In [ ]:
import torch
import torch.nn as nn
import tqdm
import numpy as np
import torch.optim as optim
import torchvision
from torchvision.utils import make_grid, save_image
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
torch.cuda.get_device_name()

'Tesla P100-PCIE-16GB'

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, img_channels, features_d):
        super().__init__()
        self.d = nn.Sequential(nn.Conv2d(img_channels, features_d, kernel_size=4, stride=2, padding=1), 
                                nn.LeakyReLU(0.2), 

                                self.conv_block(features_d, features_d * 2, 4, 2, 1), 
                                self.conv_block(features_d * 2, features_d * 4, 4, 2, 1), 
                                self.conv_block(features_d * 4, features_d * 8, 4, 2, 1), 

                                nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
                                nn.Sigmoid())
        
    def conv_block(self, in_channels, out_channels, kernel_size, stride, padding):
         return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False), 
                              nn.BatchNorm2d(out_channels), 
                              nn.LeakyReLU(0.2))

    def forward(self, x):
        return self.d(x)

In [ ]:
class Generator(nn.Module):
    def __init__(self, noise_channels, img_channels, features_g):
        super().__init__()
        self.g = nn.Sequential(self.conv_block(noise_channels, features_g * 16, 4, 1, 0), 
                               self.conv_block(features_g * 16, features_g * 8, 4, 2, 1),
                               self.conv_block(features_g * 8, features_g * 4, 4, 2, 1), 
                               self.conv_block(features_g * 4, features_g * 2, 4, 2, 1),
                               
                               nn.ConvTranspose2d(features_g * 2, img_channels, kernel_size=4, stride=2, padding=1),
                               nn.Tanh())
        
    def conv_block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False), 
                             nn.BatchNorm2d(out_channels), 
                             nn.ReLU())
    
    def forward(self, x):
        return self.g(x)

In [ ]:
def init_parameters(net):
    for n in net.modules():
        if isinstance(n, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(n.weight.data, 0.0, 0.02)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lr = 2e-4
batch_size = 128
img_size = 64
img_channels = 3
noise_dim = 100
epochs = 100
features_d = 64
features_g = 64

In [ ]:
device

device(type='cuda')

In [ ]:
transform = transforms.Compose([transforms.Resize((img_size, img_size)),
                                 transforms.ToTensor(), 
                                 transforms.Normalize([0.5 for _ in range(img_channels)], [0.5 for _ in range(img_channels)])])

In [ ]:
dataset = datasets.ImageFolder(root="faces/", transform=transform)

In [ ]:
len(dataset)

15167

In [ ]:
dataloader = DataLoader(dataset, batch_size, shuffle=True)

In [ ]:
len(dataloader)

119

In [ ]:
G = Generator(noise_dim, img_channels, features_g).to(device)
D = Discriminator(img_channels, features_d).to(device)
init_parameters(G)
init_parameters(D)

In [ ]:
opt_G = optim.Adam(G.parameters(), lr, betas=(0.5, 0.999))
opt_D = optim.Adam(D.parameters(), lr, betas=(0.5, 0.999))
loss_fn = nn.BCELoss()

In [ ]:
fixed_noise = torch.randn(64, noise_dim, 1, 1).to(device)

In [ ]:
def save_fake_images(epoch_num):
    with torch.no_grad():
        fake = G(fixed_noise)
    img_grid = make_grid(fake, normalize=True)
    file_name = "fake_images-{0:0=4d}.png".format(epoch_num)
    save_image(fake, file_name, nrow=8)
    print("")
    print("Image Saved!")
    print("")

In [ ]:
for real, _ in dataloader:
    img_grid = make_grid(real[:64], normalize=True)
    save_image(img_grid, "real_images.png", nrow=8)
    break

In [ ]:
save_fake_images(0)

In [ ]:
for epoch in range(200, 500):
    batch_losses_D = []
    batch_losses_G = []
    for real, _ in tqdm.tqdm(dataloader, total=len(dataloader)):
        real = real.to(device)
        noise = torch.randn(batch_size, noise_dim, 1, 1).to(device)
        fake = G(noise)

        D_real = D(real).reshape(-1)
        loss_D_real = loss_fn(D_real, torch.ones_like(D_real))

        D_fake = D(fake.detach()).reshape(-1)
        loss_D_fake = loss_fn(D_fake, torch.zeros_like(D_fake))

        loss_D = (loss_D_real + loss_D_fake) / 2

        D.zero_grad()
        loss_D.backward()
        opt_D.step()
        batch_losses_D.append(loss_D.item())

        output = D(fake).reshape(-1)
        loss_G = loss_fn(output, torch.ones_like(output))
        G.zero_grad()
        loss_G.backward()
        opt_G.step()
        batch_losses_G.append(loss_G.item())

    print(f" Epoch: {epoch} | Loss D: {np.round(sum(batch_losses_D)/len(batch_losses_D), 4)} | Loss G: {np.round(sum(batch_losses_G)/len(batch_losses_G), 4)}")
    if epoch % 10 == 0:
        save_fake_images(epoch)

  0%|          | 0/119 [00:00<?, ?it/s]

 Epoch: 200 | Loss D: 0.0862 | Loss G: 4.799

Image Saved!



  0%|          | 0/119 [00:00<?, ?it/s]

 Epoch: 201 | Loss D: 0.0392 | Loss G: 4.8008


  0%|          | 0/119 [00:00<?, ?it/s]

...

 Epoch: 302 | Loss D: 0.0402 | Loss G: 5.5


  0%|          | 0/119 [00:00<?, ?it/s]

 Epoch: 303 | Loss D: 0.0165 | Loss G: 5.723


 93%|█████████▎| 111/119 [00:48<00:03,  2.24it/s]

KeyboardInterrupt: ignored